In [1]:
from core import *

In [2]:
bank1 = Bank()
bank2 = Bank()

In [3]:
# Поэтапное создание клиентов
client1 = Client(bank1, "Иван", "Иванов")
client1.address = "Москва"
client1.passport = "1234 567890"

client1Facade = ClientFacade(client1)
debit_1 = client1Facade.create_account(DebitAccount)

client2 = Client(bank2, "Петр", "Петров")
client2Facade = ClientFacade(client2)
debit_2 = client2Facade.create_account(DebitAccount)

In [4]:
# Внесение наличных

client1Facade.deposit(debit_1, 1000)
debit_1.balance

1000

In [5]:
# Перевод

t1 = Transaction(debit_1, debit_2, 500).perform()
debit_1.balance, debit_2.balance

(500, 500)

In [6]:
# Альтернативно, клиенты могут через фасад переводить по id счёта получателя

debit_2.id

UUID('d14e7b62-8f10-47cb-a4ab-c0e185da49af')

In [7]:
t2 = client1Facade.transfer(debit_1, debit_2.id, 500, bank2)
debit_1.balance, debit_2.balance

(0, 1000)

In [8]:
# Если просто попросить Петрова вернуть деньги, то он не сможет, т.к. у него нет паспорта

Transaction(debit_2, debit_1, 500).perform()

ValueError: Client doesn't have passport or address


In [9]:
# Но поможет механизм отмены транзакции
t1.cancel()
t2.cancel()
debit_1.balance, debit_2.balance

(1000, 0)

In [10]:
# История транзакций
[f"{t.datetime}: {t.amount}" for t in debit_1.history.transactions]

['2023-03-15 15:32:33.187500: -500',
 '2023-03-15 15:32:33.144285: -500',
 '2023-03-15 15:32:38.723992: 500',
 '2023-03-15 15:32:33.122346: 1000',
 '2023-03-15 15:32:38.724037: 500']

In [ ]:
# Сумма всех чисел равна конечному балансу